In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
BASE_PATH = 'model_dataset'
TRAIN = 'training'
TEST = 'testing'
VAL = 'validation'
BATCH_SIZE = 15
TRAIN_PATH = os.path.join(BASE_PATH,TRAIN)
VAL_PATH = os.path.join(BASE_PATH, VAL)
TEST_PATH = os.path.join(BASE_PATH, TEST)
totalTrain = len(list(paths.list_images(TRAIN_PATH)))
totalVal = len(list(paths.list_images(VAL_PATH)))
totalTest = len(list(paths.list_images(TEST_PATH)))

In [ ]:
trainAug = ImageDataGenerator(rotation_range=25,
                              zoom_range=0.1,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.2,
                              horizontal_flip=True,
                              fill_mode="nearest")

valAug = ImageDataGenerator()
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [ ]:
# initialize the training generator
trainGen = trainAug.flow_from_directory(TRAIN_PATH,
                                        class_mode="categorical",
                                        target_size=(224, 224),
                                        color_mode="rgb",
                                        shuffle=True,
                                        batch_size=BATCH_SIZE)
# initialize the validation generator
valGen = valAug.flow_from_directory(VAL_PATH,
                                    class_mode="categorical",
                                    target_size=(224, 224),
                                    color_mode="rgb",
                                    shuffle=False,
                                    batch_size=BATCH_SIZE)
# initialize the testing generator
testGen = valAug.flow_from_directory(TEST_PATH,
                                     class_mode="categorical",
                                     target_size=(224, 224),
                                     color_mode="rgb",
                                     shuffle=False,
                                     batch_size=BATCH_SIZE)

In [ ]:
print(testGen.class_indices.keys())

In [ ]:
print("[INFO] preparing model...")
baseModel = ResNet50(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = GlobalAveragePooling2D()(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(27, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers:
    layer.trainable = False

In [ ]:
INIT_LR = 1e-4
NUM_EPOCHS = 12
opt = Adam(lr=INIT_LR, decay=INIT_LR / NUM_EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])
print("[INFO] training model...")
H = model.fit_generator(
    trainGen,
    steps_per_epoch=totalTrain // BATCH_SIZE,
    validation_data=valGen,
    validation_steps=totalVal // BATCH_SIZE,
    epochs=NUM_EPOCHS)

In [ ]:
MODEL_PATH = os.path.sep.join(["output","resnet1"])
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(totalTest // BATCH_SIZE) + 1)

predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))
print("[INFO] saving model...")
model.save(MODEL_PATH, save_format="h5")

In [ ]:
MODEL_PATH = os.path.sep.join(["output","resnet1"])
model = load_model(MODEL_PATH)
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(totalTest // BATCH_SIZE) + 1)

predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas
cm = confusion_matrix(testGen.classes, predIdxs)

In [ ]:
sns.set(font_scale=1)
plt.figure(figsize = (12,8))
sns.heatmap(cm,annot = True,cmap='PuBu')

In [ ]:
model.summary()